In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 14.0MB/s 


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 8.9MB/s 


In [0]:
import nltk
import sklearn_crfsuite
import eli5

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
import spacy
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
nlp = spacy.load("en_core_web_sm") 

In [0]:
%cd /content/drive/My Drive/WCL_tagging

/content/drive/My Drive/WCL_tagging


In [0]:
df = pd.read_csv('manual_task2_wcl.csv')
test = pd.read_csv('auto_task2_wcl.csv')
df.head()
test.head()

,Sentence,Label
0,The AK-47 ( contraction of Russian : Автомат К...,RGET RGET RGET RGET RGET RGET RGET RGET RGET R...
1,American Standard Code for Information Interch...,RGET RGET RGET RGET RGET RGET RGET RGET RGET V...
2,"The Aar ( German Aare ) , a tributary of the R...",RGET RGET RGET RGET RGET RGET RGET RGET RGET R...
3,"The Aardvark ( Orycteropus afer ) ( "" Digging ...",RGET RGET RGET RGET RGET RGET RGET RGET RGET R...
4,The Aardwolf ( Proteles cristatus ) is a small...,RGET RGET RGET RGET RGET RGET VERB GENUS GENUS...


,Sentence,Label
0,Anarchism is generally defined as the politica...,RGET VERB VERB VERB GENUS GENUS HYPER HYPER GE...
1,Autism is a disorder of neural development ch...,RGET VERB GENUS HYPER HYPER HYPER HYPER GENUS ...
2,A ( ) is the first letter and a vowel in the ...,RGET RGET RGET VERB GENUS GENUS HYPER GENUS RE...
3,Abraham Lincoln ( ) was the 16th President of ...,RGET RGET RGET RGET VERB GENUS HYPER HYPER HYP...
4,Aristotle ( ) ( ) was a Greek philosopher and ...,RGET RGET RGET RGET RGET VERB GENUS HYPER GENU...


In [0]:
df.shape

(1872, 2)

In [0]:
from sklearn.model_selection import train_test_split
train, dev = train_test_split(df, test_size= 0.2, random_state=0)
train.shape
dev.shape

(1497, 2)

(375, 2)

In [0]:
test.dropna(inplace =True)

In [0]:
test.isna().sum()

Sentence    0
Label       0
dtype: int64

In [0]:
train['Sentence'] = train['Sentence'].apply(lambda x : x.strip())
dev['Sentence'] = dev['Sentence'].apply(lambda x: x.strip())
test['Sentence'] = test['Sentence'].apply(lambda x:x.strip())
train.head()
dev.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Sentence,Label
966,The Lhop or Doya are a little-known tribe of ...,RGET RGET RGET RGET VERB GENUS GENUS HYPER GEN...
1189,"An Ocean ( from Greek Ὠκεανὸς , Okeanos ( Ocea...",RGET RGET RGET RGET RGET RGET RGET RGET RGET R...
1093,"Richard Melville Hall , also known as Moby ( b...",RGET RGET RGET RGET RGET RGET RGET RGET RGET R...
1223,The PC-FX is a video game console released in...,RGET RGET VERB GENUS HYPER HYPER HYPER GENUS R...
1421,The Quasi-War was an undeclared war fought en...,RGET RGET VERB GENUS HYPER HYPER GENUS REST RE...


,Sentence,Label
1579,"A Signature ( from Latin signare , "" sign "" ) ...",RGET RGET RGET RGET RGET RGET RGET RGET RGET R...
254,"The Bundestag ( "" Federal Diet "" or "" Lower Ho...",RGET RGET RGET RGET RGET RGET RGET RGET RGET R...
1794,a CRUSTACEAN is any mainly aquatic arthropod ...,RGET RGET VERB VERB GENUS GENUS HYPER GENUS RE...
85,The Amiga is a family of personal computers o...,RGET RGET VERB VERB VERB VERB GENUS HYPER GENU...
1286,The Phanerozoic ( occasionally Phanaerozoic ) ...,RGET RGET RGET RGET RGET RGET RGET VERB GENUS ...


In [0]:
from tqdm import tqdm_notebook as tqdm
def pos_tagger(df):
  df_pos_tags = []
  for i, val in tqdm(df.iterrows()):
    doc = nlp(df['Sentence'][i])
    pos_tags = []
    for token in doc:
      pos_tags.append(token.pos_)
    pos_tags = ' '.join(pos_tags)
    df_pos_tags.append(pos_tags)
  df['POS']= df_pos_tags
pos_tagger(train)
pos_tagger(dev)
pos_tagger(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


KeyboardInterrupt: ignored

In [0]:
train.head()

,Sentence,Label,POS
966,The Lhop or Doya are a little-known tribe of ...,RGET RGET RGET RGET VERB GENUS GENUS HYPER GEN...,DET PROPN CCONJ PROPN AUX DET ADV PUNCT VERB N...
1189,"An Ocean ( from Greek Ὠκεανὸς , Okeanos ( Ocea...",RGET RGET RGET RGET RGET RGET RGET RGET RGET R...,DET PROPN PUNCT ADP PROPN PROPN PUNCT PROPN PU...
1093,"Richard Melville Hall , also known as Moby ( b...",RGET RGET RGET RGET RGET RGET RGET RGET RGET R...,PROPN PROPN PROPN PUNCT ADV VERB SCONJ PROPN P...
1223,The PC-FX is a video game console released in...,RGET RGET VERB GENUS HYPER HYPER HYPER GENUS R...,DET PROPN PUNCT PROPN AUX DET NOUN NOUN NOUN S...
1421,The Quasi-War was an undeclared war fought en...,RGET RGET VERB GENUS HYPER HYPER GENUS REST RE...,DET PROPN PUNCT PROPN AUX DET ADJ NOUN SPACE V...


In [0]:
test['Label'] = test['Sentence'].apply(lambda x: ' '.join(['O']*len(x.split())))
test['Label']

In [0]:
train.index.values

array([ 966, 1189, 1093, ..., 1653,  559,  684])

In [0]:
def data_prep(df):
  data = []
  for i in tqdm(df.index.values):
      # Take the word, POS tag, and its label
      words= df['Sentence'][i].split()
      pos =df['POS'][i].split()
      label = df['Label'][i].split()
      data.append([(w, pos, label) for (w, pos, label) in zip(words, pos, label)])
  return data
train_prep = data_prep(train)
dev_prep = data_prep(dev)
# test_prep = data_prep(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
train_prep[0]

[('The', 'DET', 'RGET'),
 ('Lhop', 'PROPN', 'RGET'),
 ('or', 'CCONJ', 'RGET'),
 ('Doya', 'PROPN', 'RGET'),
 ('are', 'AUX', 'VERB'),
 ('a', 'DET', 'GENUS'),
 ('little-known', 'ADV', 'GENUS'),
 ('tribe', 'PUNCT', 'HYPER'),
 ('of', 'VERB', 'GENUS'),
 ('southwest', 'NOUN', 'REST'),
 ('Bhutan', 'SPACE', 'REST'),
 ('.', 'ADP', 'REST')]

In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

X_train = [sent2features(s) for s in train_prep]
y_train = [sent2labels(s) for s in train_prep]

X_test = [sent2features(s) for s in dev_prep]
y_test = [sent2labels(s) for s in dev_prep]

In [0]:
X_train[0][1]

{'+1:postag': 'CCONJ',
 '+1:postag[:2]': 'CC',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'or',
 '-1:postag': 'DET',
 '-1:postag[:2]': 'DE',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:word.lower()': 'the',
 'bias': 1.0,
 'postag': 'PROPN',
 'postag[:2]': 'PR',
 'word.isdigit()': False,
 'word.istitle()': True,
 'word.isupper()': False,
 'word.lower()': 'lhop',
 'word[-3:]': 'hop'}

In [0]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 3.5 s, sys: 6.91 ms, total: 3.5 s
Wall time: 3.51 s


In [0]:
# eli5.show_weights(crf, top=30)

In [0]:
labels = list(crf.classes_)
labels

['RGET', 'VERB', 'GENUS', 'HYPER', 'REST']

In [0]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

0.9290952798184414

In [0]:
#For the test set and final submission

# X_test_final = [sent2features(s) for s in test_prep]
# y_test_final = [sent2labels(s) for s in test_prep]

# y_pred_final = crf.predict(X_test_final)

# test['Label'] =  pd.DataFrame([' '.join(x) for x in y_pred_final])
# test.head()

# test[['Sentence','Label']].to_csv('submit_task2_test.csv', index=False)

In [0]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       GENUS      0.855     0.748     0.798      1297
        VERB      0.963     0.900     0.930       838
        REST      0.935     0.980     0.957      4894
        RGET      0.966     0.994     0.980      2516
       HYPER      0.830     0.682     0.749       529

    accuracy                          0.932     10074
   macro avg      0.910     0.861     0.883     10074
weighted avg      0.929     0.932     0.929     10074



In [0]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=200,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X_train, y_train)

y_pred = crf.predict(X_test)
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=200, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

              precision    recall  f1-score   support

       GENUS      0.662     0.446     0.533      1297
        VERB      0.933     0.379     0.539       838
        REST      0.775     0.956     0.856      4894
        RGET      0.877     0.893     0.885      2516
       HYPER      0.731     0.359     0.482       529

    accuracy                          0.795     10074
   macro avg      0.795     0.607     0.659     10074
weighted avg      0.797     0.795     0.776     10074



In [0]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=True,
)
crf.fit(X_train, y_train);

y_pred = crf.predict(X_test)
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

              precision    recall  f1-score   support

       GENUS      0.804     0.692     0.744      1297
        VERB      0.889     0.857     0.872       838
        REST      0.898     0.976     0.935      4894
        RGET      0.974     0.960     0.967      2516
       HYPER      0.781     0.520     0.624       529

    accuracy                          0.902     10074
   macro avg      0.869     0.801     0.829     10074
weighted avg      0.898     0.902     0.897     10074



In [0]:
eli5.show_weights(crf, top=10, targets=['RGET','GENUS','VERB','REST','HYPER'])

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [0]:
expl = eli5.explain_weights(crf, top=5, targets=['RGET','GENUS','VERB','REST','HYPER'])
print(eli5.format_as_text(expl))

Explained as: CRF

Transition features:
         RGET    GENUS    VERB    REST    HYPER
-----  ------  -------  ------  ------  -------
RGET    2.389   -6.533   0.263  -6.352   -4.367
GENUS  -6.500    1.747  -4.129  -0.205    1.145
VERB   -4.862    0.433   2.157  -5.602   -3.310
REST   -6.960   -6.495  -6.700   3.127   -6.106
HYPER  -4.271    0.763  -3.121  -6.273    1.284

y='RGET' top features
Weight  Feature           
------  ------------------
+6.165  BOS               
+2.844  +1:word.lower():is
  … 4990 more positive …  
  … 950 more negative …   
-1.035  -1:word.lower():is
-1.253  word[-3:]:is      
-1.253  word.lower():is   

y='GENUS' top features
Weight  Feature         
------  ----------------
+1.931  word[-3:]:ER>   
 … 4718 more positive … 
 … 884 more negative …  
-1.713  -1:postag:SPACE 
-1.713  -1:postag[:2]:SP
-1.745  +1:postag[:2]:SP
-1.745  +1:postag:SPACE 

y='VERB' top features
Weight  Feature           
------  ------------------
+1.847  -1:word.lower():) 
+1.06

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [0]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=5,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  3.1min
[Parallel(n_jobs=5)]: Done 150 out of 150 | elapsed: 11.2min finished


CPU times: user 11min, sys: 2.38 s, total: 11min 2s
Wall time: 11min 15s


In [0]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.5776144095075597, 'c2': 0.001981312654521531}
best CV score: 0.9254381594433871
model size: 0.22M


In [0]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       GENUS      0.839     0.769     0.803      1297
        VERB      0.951     0.901     0.925       838
        REST      0.932     0.984     0.957      4894
        RGET      0.988     0.975     0.981      2516
       HYPER      0.822     0.682     0.746       529

    accuracy                          0.932     10074
   macro avg      0.906     0.862     0.883     10074
weighted avg      0.930     0.932     0.930     10074



In [0]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
REST   -> REST    3.308326
VERB   -> VERB    2.838809
RGET   -> RGET    2.563495
GENUS  -> GENUS   1.629476
HYPER  -> HYPER   1.547567
VERB   -> GENUS   0.958174
HYPER  -> GENUS   0.870438
GENUS  -> HYPER   0.593812
GENUS  -> REST    0.266533
RGET   -> HYPER   -4.535157
VERB   -> REST    -5.703113
VERB   -> HYPER   -5.788373
RGET   -> REST    -6.905487
VERB   -> RGET    -6.999849
RGET   -> GENUS   -7.362438
HYPER  -> RGET    -7.794159
GENUS  -> VERB    -8.187645
HYPER  -> VERB    -8.468481
GENUS  -> RGET    -8.582203
REST   -> HYPER   -8.637626

Top unlikely transitions:
HYPER  -> HYPER   1.547567
VERB   -> GENUS   0.958174
HYPER  -> GENUS   0.870438
GENUS  -> HYPER   0.593812
GENUS  -> REST    0.266533
RGET   -> HYPER   -4.535157
VERB   -> REST    -5.703113
VERB   -> HYPER   -5.788373
RGET   -> REST    -6.905487
VERB   -> RGET    -6.999849
RGET   -> GENUS   -7.362438
HYPER  -> RGET    -7.794159
GENUS  -> VERB    -8.187645
HYPER  -> VERB    -8.468481
GENUS  -> R

In [0]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
6.165257 RGET     BOS
2.843826 RGET     +1:word.lower():is
1.930978 GENUS    word[-3:]:ER>
1.847113 VERB     -1:word.lower():)
1.209703 HYPER    +1:word.lower():that
1.132454 REST     -1:postag:SPACE
1.132454 REST     -1:postag[:2]:SP
1.100209 REST     -1:word.lower():that
1.061980 VERB     word.lower():is
1.061980 VERB     word[-3:]:is
1.004844 GENUS    -1:word.lower():</hyper>
0.978571 GENUS    word.lower():</hyper>
0.951250 GENUS    word.lower():<hyper>
0.941102 GENUS    +1:word.lower():</hyper>
0.927661 GENUS    -1:word.lower():<hyper>
0.856918 RGET     word[-3:]:In
0.771091 HYPER    word[-3:]:ion
0.751842 REST     EOS
0.743940 VERB     word[-3:]:of
0.743696 VERB     word.lower():of
0.728766 VERB     +1:word.lower():a
0.717078 GENUS    -1:word.lower():of
0.708971 RGET     word.lower():in
0.698632 VERB     postag:VERB
0.698632 VERB     postag[:2]:VE
0.694526 REST     word.lower():.
0.694526 REST     word[-3:]:.
0.690274 HYPER    -1:postag:SPACE
0.690274 HYPER    -1:pos